In [4]:
import chemistry_psql as cp
import importlib
importlib.reload(cp)
import psycopg2
import pandas as pd

#dfm = pd.read_csv("../RBP_Manual.csv")
#dfm = pd.read_csv("../data-1633411346295.csv")
#print(type(dfm))
dfm = pd.read_csv("../Splicng_agents.csv")
print(dfm)

struct_dict = ['Smiles','canonical_smiles','smiles', 'SMILES']
#print(dfm.columns)
#print()
#struct_col = set(struct_dict).intersection(dfm.columns)
struct_col = list(set(struct_dict).intersection(dfm.columns))[0]
print('Structure Column : ',struct_col)

struct = dfm[[struct_col]].drop_duplicates()
#print(struct)
#struct = struc.drop_duplicates()
#print(struct)

struct_list = struct.to_records(index = False)
#print(struct_list)

imp_struct = ()

print()
print('# # # Structure Insert # # #')
print()
for i in struct_list:
    #print(i)
    cp.SmilesInsert(i)


print()
print('# # # Structure Search # # #')
print()
db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
#db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
for i in struct_list:
    j= i[0]
    print()
    print(j)
    sql = 'select * from public.structures where molecule =\' '+j+' \'::mol;'
    #print(sql)
    cur.execute(sql)
    mollist = cur.fetchall()
    print(mollist)



#test = ("C1CCCCOCCCC1","C1CCCCNCCCC1","C1CCCCSCCCC1")

#cp.SmilesInsert(test)

                                               SMILES                 Name
0              ClC1=CC(NC(C2=NOC(C(C)C)=C2)=O)=CC=C1F            LDN-75654
1                FC1=CC=C(NC(C2=NOC(C(C)C)=C2)=O)C=C1  jmedchem_6b01885_4a
2               ClC1=CC(NC(C2=NOC(C(C)C)=C2)=O)=CC=C1  jmedchem_6b01885_4b
3                   O=C(C1=NOC(C(C)C)=C1)NC2=CC=CC=C2  jmedchem_6b01885_4c
4                  O=C(C1=NOC(C(C)C)=C1)NC2=CC=CC=C2F  jmedchem_6b01885_4d
..                                                ...                  ...
59  [H][C@]1(C)CN(CCN1)C1=CC2=C(C=C1)C=C(C1=CN3C=C...     Compound 1 (SMN)
60  CN1CCN(CC1)C1=CC2=C(C=C1)N=C(N=C2)C1=CN2C=C(C)...               SMN-CX
61   CC1=CN2C=C(C=C(F)C2=N1)C1=CC(=O)N2C=C(F)C=CC2=N1             SMN-Core
62  CCN(CC)C1=CC2=C(C=C1)C=C(C1=NC3=C(O1)C=CC=C3)C...        SMN-Hit-Roche
63  [H][C@@]1(CC)CN(CCN1)C1=CC=C(C(=O)NC2=CN3C=C(C...               SMN-CY

[64 rows x 2 columns]
Structure Column :  SMILES

# # # Structure Insert # # #

Structure Exist:  C

[(15681, 'CCc1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 'CCc1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 13314)]

O=C(C1=NOC(CC(C)C)=C1)NC2=CC(Cl)=C(F)C=C2
[(15662, 'CC(C)Cc1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 'CC(C)Cc1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 14231)]

O=C(C1=NOC(C(C)(C)C)=C1)NC2=CC(Cl)=C(F)C=C2
[(15676, 'CC(C)(C)c1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 'CC(C)(C)c1cc(C(=O)Nc2ccc(F)c(Cl)c2)no1', 14701)]

O=C(C1=NOC(C2CCCCC2)=C1)NC3=CC(Cl)=C(F)C=C3
[(16524, 'O=C(C1=NOC(C2CCCCC2)=C1)NC3=CC(Cl)=C(F)C=C3', 'O=C(Nc1ccc(F)c(Cl)c1)c1cc(C2CCCCC2)on1', None)]

O=C(C1=NOC(C2CCCC2)=C1)NC3=CC(Cl)=C(F)C=C3
[(16525, 'O=C(C1=NOC(C2CCCC2)=C1)NC3=CC(Cl)=C(F)C=C3', 'O=C(Nc1ccc(F)c(Cl)c1)c1cc(C2CCCC2)on1', None)]

O=C(C1=NOC(C2=CC=CC=C2)=C1)NC3=CC(Cl)=C(F)C=C3
[(16526, 'O=C(C1=NOC(C2=CC=CC=C2)=C1)NC3=CC(Cl)=C(F)C=C3', 'O=C(Nc1ccc(F)c(Cl)c1)c1cc(-c2ccccc2)on1', None)]

O=C(C1=NOC(CC2=CC=CC=C2)=C1)NC3=CC(Cl)=C(F)C=C3
[(15663, 'O=C(Nc1ccc(F)c(Cl)c1)c1cc(Cc2ccccc2)on1', 'O=C(Nc1ccc(F)c(Cl)c1)c1cc(Cc2ccccc2)on1', 13754)]

O=C(C1=NOC(C(C)C)=

In [ ]:
from rdkit import Chem
from rdkit.Chem import SaltRemover as rdsr

remover = rdsr.SaltRemover()

# We can have a look at some of the salts
a = Chem.MolToSmarts(remover.salts[3])
print(a)
print(len(remover.salts))

for i in remover.salts:
    a = Chem.MolToSmarts(i)
    print(a)

In [ ]:
from rdkit.Chem.MolStandardize import rdMolStandardize
un = rdMolStandardize.Uncharger()

for i in struct_list:
    j = Chem.MolFromSmiles(i[0])
    r = remover.StripMol(j)
    if j.GetNumAtoms() != r.GetNumAtoms():
        r = un.uncharge(r)
        k = Chem.MolToSmiles(r)
        print(i[0], ' - ',k)

In [2]:
import sys
sys.path

['/mnt/raid1/Python_projects/Neural_Network',
 '/home/peter/anaconda3/envs/rdkit/lib/python37.zip',
 '/home/peter/anaconda3/envs/rdkit/lib/python3.7',
 '/home/peter/anaconda3/envs/rdkit/lib/python3.7/lib-dynload',
 '',
 '/home/peter/anaconda3/envs/rdkit/lib/python3.7/site-packages',
 '/home/peter/anaconda3/envs/rdkit/lib/python3.7/site-packages/IPython/extensions',
 '/home/peter/.ipython']

In [3]:
sys.path.append('../pkg_mod')